# Sparkify Project Workspace
This workspace contains a tiny subset (128MB) of the full dataset available (12GB). Feel free to use this workspace to build your project, or to explore a smaller subset with Spark before deploying your cluster on the cloud. Instructions for setting up your Spark cluster is included in the last lesson of the Extracurricular Spark Course content.

You can follow the steps below to guide your data analysis and model building portion of this project.

In [ ]:
# import libraries
# 1) import any other libraries you might need
from pyspark.sql import SparkSession
from pyspark.sql.functions import avg, desc, min, max, split, udf
from pyspark.sql.types import IntegerType

from pyspark.ml import Pipeline
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.ml.feature import CountVectorizer, StandardScaler, StringIndexer, VectorAssembler, OneHotEncoder
from pyspark.ml.regression import LinearRegression
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder

import re

In [ ]:
# create a Spark session
spark = SparkSession.builder\
                    .appName('Sparkify')\
                    .getOrCreate()

# Load and Clean Dataset
In this workspace, the mini-dataset file is `mini_sparkify_event_data.json`. Load and clean the dataset, checking for invalid or missing data - for example, records without userids or sessionids. 

In [ ]:
# I cannot upload the dataset in github so if you want to download or use it you can use this link: 
# https://video.udacity-data.com/topher/2018/December/5c1d6681_medium-sparkify-event-data/medium-sparkify-event-data.json
df = spark.read.json("mini_sparkify_event_data.json")

# Exploratory Data Analysis
When you're working with the full dataset, perform EDA by loading a small subset of the data and doing basic manipulations within Spark. In this workspace, you are already provided a small subset of data you can explore.



In [ ]:
df.printSchema()

In [ ]:
df.head()

In [ ]:
df.select('userId').show()

### Deleting any useless rows

In [ ]:
df = df.where("userId != '' and userId != ' '")

In [ ]:
df = df.dropna(how='any', subset=['userId', 'sessionId'])

In [ ]:
valid = df.dropDuplicates()

In [ ]:
valid.groupby('userId').max().select('userId').sort('userId').show()

### Define Churn

Once you've done some preliminary analysis, create a column `Churn` to use as the label for your model. I suggest using the `Cancellation Confirmation` events to define your churn, which happen for both paid and free users. As a bonus task, you can also look into the `Downgrade` events.

### Explore Data
Once you've defined churn, perform some exploratory data analysis to observe the behavior for users who stayed vs users who churned. You can start by exploring aggregates on these two groups of users, observing how much of a specific action they experienced per a certain time unit or number of songs played.

In [ ]:
valid.select('page').dropDuplicates().sort('page').show()

We have the following values that are in the `page` column.

* Add Friend
* Add to Playlist
* Cancel
* Cancellation Confirmation
* Downgrade
* Error
* Help
* Home
* Logout
* NextSong
* Roll Advert
* Save Settings
* Settings
* Submit Downgrade
* Submit Upgrade
* Thumbs Down
* Thumbs Up
* Upgrade<br>
The pages we never want to see users utilize is the **Cancel** and especially the `Cancellation Confirmation`. </p>

In [ ]:
churn = udf(lambda x: 1 if x == 'Cancellation Confirmation' else 0, IntegerType())

In [ ]:
valid = valid.withColumn('churn', churn(valid.page))

In [ ]:
valid.where("churn == 1").count()

# Feature Engineering
Once you've familiarized yourself with the data, build out the features you find promising to train your model on. To work with the full dataset, you can follow the following steps.
- Write a script to extract the necessary features from the smaller subset of data
- Ensure that your script is scalable, using the best practices discussed in Lesson 3
- Try your script on the full data set, debugging your script if necessary

If you are working in the classroom workspace, you can just extract features based on the small subset of data contained here. Be sure to transfer over this work to the larger dataset when you work on your Spark cluster.

In [ ]:
valid.groupby(["churn", 'gender']).count().show()

In [ ]:
gender_idx = StringIndexer(inputCol="gender", outputCol='gender_idx')
page_idx = StringIndexer(inputCol="page", outputCol='page_idx')
gender_encoder = OneHotEncoder(inputCol='gender_idx', outputCol='gender_vec')
page_encoder = OneHotEncoder(inputCol='page_idx', outputCol='page_vec')
assembler = VectorAssembler(inputCols=["gender_vec", "page_vec", "status"], outputCol="features")
idx = StringIndexer(inputCol="churn", outputCol="label")

# Modeling
Split the full dataset into train, test, and validation sets. Test out several of the machine learning methods you learned. Evaluate the accuracy of the various models, tuning parameters as necessary. Determine your winning model based on test accuracy and report results on the validation set. Since the churned users are a fairly small subset, I suggest using F1 score as the metric to optimize.

In [ ]:
model =  LogisticRegression(maxIter=15)
pipeline = Pipeline(stages=[gender_idx, page_idx, gender_encoder, page_encoder, assembler, idx, model])

> I chose `LogisticRegression` as we want a simple `1` or `0` prediction <br>
> now let's split our data

In [ ]:
rest, validation = valid.randomSplit([0.9, 0.1], seed=42)

> Building the pipeline

In [ ]:
param = ParamGridBuilder() \
    .addGrid(model.regParam,[0.0, 0.1, ]) \
    .addGrid(model.elasticNetParam,[0.0, 0.1, 0.5, 1.0]) \
    .build()


cross = CrossValidator(estimator=pipeline,
                          estimatorParamMaps=param,
                          evaluator=MulticlassClassificationEvaluator(),
                          numFolds=3)

### Fit the Data

In [ ]:
fitted = cross.fit(rest)

In [ ]:
results = fitted.transform(validation)

In [ ]:
results.select("rawPrediction","probability", "prediction").show()

In [ ]:
fitted.avgMetrics

We can see that we have an awesome score accuracy about 0.99 with 15 iterations!

# Final Steps
Clean up your code, adding comments and renaming variables to make the code easier to read and maintain. Refer to the Spark Project Overview page and Data Scientist Capstone Project Rubric to make sure you are including all components of the capstone project and meet all expectations. Remember, this includes thorough documentation in a README file in a Github repository, as well as a web app or blog post.

**Done**